## Imports

In [2]:
import os
import time
import math

import cv2
import numpy as np
import imutils

from server.helpers.estimator import TfPoseEstimator
from server.helpers.networks import get_graph_path, model_wh
from server.helpers.calculate_angle import calculate_angle

from IPython.display import clear_output

D:\Softwares\Anaconda\envs\tensor_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Softwares\Anaconda\envs\tensor_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Softwares\Anaconda\envs\tensor_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Softwares\Anaconda\envs\tensor_gpu\lib\site-packages\tensorflow\

## Constants

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=14)
def pdictstr(d):
    return "{" + pp.pformat(DATA_PATHS)[14:]
    
print("CONSTANTS:")

# Paths
BASE_DIR = os.path.abspath('.')
GRAPH_PATH = os.path.join(BASE_DIR, 'server', 'models', 'mobilenet', 'graph_opt.pb')
DATA_PATHS = {'base': os.path.join(BASE_DIR, 'data'),
              'sample': os.path.join(BASE_DIR, 'data', 'sample')}

print("BASE_DIR =", BASE_DIR)
print("GRAPH_PATH =", GRAPH_PATH)
print("DATA_PATHS =", pdictstr(DATA_PATHS))

# Processing
PROCESSING_DIMS = (240, 240)

print("PROCESSING_DIMS =", PROCESSING_DIMS)

# Camera
CAMERA_NUMBER = 1

print("CAMERA_NUMBER =", CAMERA_NUMBER)

CONSTANTS:
BASE_DIR = C:\Users\USER\Documents\phormatics
GRAPH_PATH = C:\Users\USER\Documents\phormatics\server\models\mobilenet\graph_opt.pb
DATA_PATHS = {'base': 'C:\\Users\\USER\\Documents\\phormatics\\data',
              'sample': 'C:\\Users\\USER\\Documents\\phormatics\\data\\sample'}
PROCESSING_DIMS = (240, 240)
CAMERA_NUMBER = 1


In [4]:
def average_or_one(body_parts, idx1, idx2):
    if idx1 in body_parts.keys() and idx2 in body_parts.keys():
        return ((body_parts[idx1].x + body_parts[idx2].x)/2, (body_parts[idx1].y + body_parts[idx2].y)/2) 
    elif idx1 in body_parts.keys():
        return (body_parts[idx1].x, body_parts[idx1].y)
    elif idx2 in body_parts.keys():
        return (body_parts[idx2].x, body_parts[idx2].y)
    else: 
        return False

# Initialize fps counter
fps_time = 0

# Load model
model = TfPoseEstimator(GRAPH_PATH, target_size=(240,240))

# Initialize camera capture
#cam = cv2.VideoCapture(CAMERA_NUMBER)
cam = cv2.VideoCapture('data/sample/curls.mp4')
ret_val, image = cam.read()

# Read loop
while True:
    # Read from camera
    # Predict poses
    
    # Get biggest human
    try:
        ret_val, image = cam.read()
        humans = model.inference(image)
        human = humans[0]
        largest_torso = 0
        for h in humans:
            try:
                # average shoulders
                shoulder = average_or_one(h.body_parts, 2, 5)
                # average hips
                hip = average_or_one(h.body_parts, 8, 11)

                print(shoulder)
                print(hip)

                if shoulder and hip:
                    torso = (hip[0] - shoulder[0])**2 + (hip[1] - shoulder[1])**2
                    torso = math.sqrt(torso)

                    if torso > largest_torso:
                        human = h
            except KeyError:
                pass


        # Draw pose
        draw = TfPoseEstimator.draw_humans(image, [human], imgcopy=False)

        # Draw fps
        fps = 1.0 / (time.time() - fps_time)
        cv2.putText(draw,
                    "FPS: {}".format(fps),
                    (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 0), 2)

        # Show image
        cv2.imshow('tf-pose-estimation result', draw)

        # Restart FPS counter
        fps_time = time.time()
        if cv2.waitKey(1) == 27:
            break
    except:
        pass

cam.release()
cv2.destroyAllWindows()

(0.4666666666666667, 0.36666666666666664)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.3333333333333333)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.35)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.35)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.3333333333333333)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.35)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.35)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.35)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.3333333333333333)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.3333333333333333)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.35)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.36666666666666664)
(0.48333333333333334, 0.7333333333333333)
(0.4666666666666667, 0.36666666666666664)
(0.48333333333333334, 0.75)
(0.4666666666666

(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7166666666666666)
(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.48333333333333334, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.5, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.48333333333333334, 0.7166666666666666)
(0.48333333333333334, 0.31666666666666665)
(0.48333333333333334, 0.7333333333333333)
(0.48333333333333334, 0.31666666666666665)
(0.48333333333333334, 0.7)
(0.48333333333333334, 0.31666666666666665)
(0.48333333333333334, 0.7166666666666666)
(0.48333333333333334, 0.3)
(0.48333333333333334, 0.7166666666666666)
(0.48333333333333334, 0.3)
(0.48333

(0.5666666666666667, 0.3)
(0.5666666666666667, 0.6666666666666666)
(0.5666666666666667, 0.3)
(0.55, 0.6666666666666666)
(0.5666666666666667, 0.3)
(0.55, 0.6499999999999999)
(0.5666666666666667, 0.3)
(0.55, 0.6499999999999999)
(0.5666666666666667, 0.3)
(0.55, 0.6499999999999999)
(0.5666666666666667, 0.3)
(0.55, 0.6666666666666666)
(0.5666666666666667, 0.3)
(0.55, 0.6499999999999999)
(0.5666666666666667, 0.31666666666666665)
(0.55, 0.6499999999999999)
(0.5666666666666667, 0.31666666666666665)
(0.55, 0.6499999999999999)
(0.5666666666666667, 0.3)
(0.55, 0.6333333333333333)
(0.5666666666666667, 0.3)
(0.55, 0.6499999999999999)
(0.5666666666666667, 0.3)
(0.55, 0.6333333333333333)
(0.5666666666666667, 0.3)
(0.55, 0.6333333333333333)
(0.5666666666666667, 0.3)
(0.5666666666666667, 0.6333333333333333)
(0.5666666666666667, 0.3)
(0.5666666666666667, 0.6333333333333333)
(0.5666666666666667, 0.3)
(0.5666666666666667, 0.6333333333333333)
(0.5666666666666667, 0.3)
(0.55, 0.6499999999999999)
(0.56666666

## Body part mapping
Nose = 0

Neck = 1

RShoulder = 2

RElbow = 3

RWrist = 4

LShoulder = 5

LElbow = 6

LWrist = 7

RHip = 8

RKnee = 9

RAnkle = 10

LHip = 11

LKnee = 12

LAnkle = 13

REye = 14

LEye = 15

REar = 16

LEar = 17

Background = 18

In [5]:
def average_or_one(body_parts, idx1, idx2):
    if idx1 in body_parts.keys() and idx2 in body_parts.keys():
        return ((body_parts[idx1].x + body_parts[idx2].x)/2, (body_parts[idx1].y + body_parts[idx2].y)/2) 
    elif idx1 in body_parts.keys():
        return (body_parts[idx1].x, body_parts[idx1].y)
    elif idx2 in body_parts.keys():
        return (body_parts[idx2].x, body_parts[idx2].y)
    else: 
        return False
    
def angle_one(body_parts, idx):
    return (body_parts[idx].x, body_parts[idx].y)
    
def analyze_workout(body_parts, side, workout_func):
    if side:
        return workout_func(body_parts, side)
    else:
        return workout_func(body_parts)

def plank(body_parts):
    """
    Problems:
    ->  Deviation in waist:
        Body Parts:
           L/R Shoulder
           L/R Hip
           L/R Ankle
        Percent Deviation:
           (OptimalAngle - AngleDetected)/OptimalAngle * 100
        Params:
            OptimalAngle = pi
            Threshold = 0.1
    """
    
    def deviation_in_waist(body_parts, optimal_angle, thresh):
        # average shoulders
        shoulder = average_or_one(body_parts, 2, 5)
        # average hips
        hip = average_or_one(body_parts, 8, 11)    
        # average ankles
        ankle = average_or_one(body_parts, 10, 13)
        try:
            # calculate angle
            angle_detected = calculate_angle(shoulder, hip, ankle)
        except TypeError as e:
            raise e
        # calculate percent deviation
        deviation = (optimal_angle - angle_detected)/optimal_angle * 100
        return deviation
    
    
    return deviation_in_waist(body_parts, math.pi, 0.1)

def curl(body_parts, side):
    """
    side - left or right, depending on user
    
    Problems:
    ->  Horizontal deviation in humerous to upper body:
        Body parts:
            L//R Shoulder
            L//R Elbow
            L//R Hip
        Percent Deviation:
            (OptimalAngle - AngleDetected)/OptimalAngle * 100
        Params:
            OptimalAngle = 0
            Threshold = 0.1
    """
    
    def horizontal_deviation_of_elbow(body_parts, side, optimal_angle, thresh):
        ankle = average_or_one(body_parts, 10, 13)
        try:
            if side == 'L':
                shoulder = angle_one(body_parts, 5)
                elbow = angle_one(body_parts, 6)
                hip = angle_one(body_parts, 11)
            elif side == 'R':
                shoulder = angle_one(body_parts, 2)
                elbow = angle_one(body_parts, 3)
                hip = angle_one(body_parts, 8)
            else:
                return -1
        except KeyError as e:
            return -1
        
        try:
            if shoulder and hip and ankle:
                # calculate angle
                angle_detected = calculate_angle(shoulder, hip, ankle)
            else:
                return -1
        except TypeError as e:
            raise e
        
        # calculate percent deviation
        deviation = (optimal_angle - angle_detected)/optimal_angle * 100
        return deviation
        
        
        
    return horizontal_deviation_of_elbow(body_parts, side, 0, 0.1)

def pushup(body_parts):
    """
    Problems:
    ->  Deviation in waist:
        Body Parts:
           L/R Shoulder
           L/R Hip
           L/R Ankle
        Percent Deviation:
           (OptimalAngle - AngleDetected)/OptimalAngle * 100
        Params:
            OptimalAngle = pi
            Threshold = 0.1
    """
    
    def deviation_in_waist(body_parts, optimal_angle, thresh):
        # average shoulders
        shoulder = average_or_one(body_parts, 2, 5)
        # average hips
        hip = average_or_one(body_parts, 8, 11)    
        # average ankles
        ankle = average_or_one(body_parts, 10, 13)
        try:
            if shoulder and hip and ankle:
                # calculate angle
                angle_detected = calculate_angle(shoulder, hip, ankle)
            else:
                return -1
        except TypeError as e:
            raise e
        # calculate percent deviation
        deviation = (optimal_angle - angle_detected)/optimal_angle * 100
        return deviation


    return deviation_in_waist(body_parts, math.pi, 0.1)

def squat(body_parts):
    """
    Problems:
    - Squat depth
        Body Parts:
           L/R Ankle
           L/R Knee
           L/R Hip
        Percent Deviation:
           (OptimalAngle - AngleDetected)/OptimalAngle * 100
        Params:
            OptimalAngle = pi/2
            Threshold = TBD
    - Forward knee movement
        Body Parts:
            L/R Ankle
            L/R Knee
        Percent Deviation:
           (X_ANKLE - X_KNEE)/TibiaLength * 100
        Params:
            OptimalDeviation = 0
            Threshold = TBD
    - 'Divebombing'
        Body Parts:
            L/R Shoulder
            L/R Hip
        Percent Deviation:
           (X_SHOULDER - X_HIP)/TorsoLength * 100
        Params:
            OptimalDeviation = 0
            Threshold = TBD
    """
    def squat_depth_angle(body_parts, optimal_angle, thresh):
        ankle = average_or_one(body_parts, 10, 13)
        knee = average_or_one(body_parts, 9, 12)
        hip = average_or_one(body_parts, 8, 11)
        try:
            if ankle and knee and hip:
                angle_detected = calculate_angle(ankle, knee, hip)
            else:
                 return -1
        except TypeError as e:
            raise e
        # calculate percent deviation
        deviation = (optimal_angle - angle_detected)/optimal_angle * 100
        return deviation
    return squat_depth_angle(body_parts, (math.pi)/2, 0.1)

                
    

## Static Test

### Plank

In [6]:
# HARDCODE
current_workout = plank

# Initialize fps counter
fps_time = 0

# Load model
model = TfPoseEstimator(GRAPH_PATH, target_size=PROCESSING_DIMS)

# Initialize camera capture
image1 = cv2.imread(os.path.join(DATA_PATHS['sample'], 'plank', 'plank-up.jpg'))
image2 = cv2.imread(os.path.join(DATA_PATHS['sample'], 'plank', 'plank-down.jpg'))

# Predict poses
humans1 = model.inference(image1)
humans2 = model.inference(image2)

# Analyze workout
dev1 = analyze_workout(humans1[0].body_parts,0, current_workout)
dev2 = analyze_workout(humans2[0].body_parts,0, current_workout)

# Draw pose
draw1 = TfPoseEstimator.draw_humans(image1, humans1, imgcopy=False)
draw2 = TfPoseEstimator.draw_humans(image2, humans2, imgcopy=False)

# Draw angle
cv2.putText(draw1,
                "Deviation: {}".format(dev1),
                (10, 40),  cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0), 2)

cv2.putText(draw2,
                "Deviation: {}".format(dev2),
                (10, 40),  cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0), 2)

# Read loop
while True:
    
    # Show image
    cv2.imshow('tf-pose-estimation result1', draw1)
    cv2.imshow('tf-pose-estimation result2', draw2)

    # Restart FPS counter
    fps_time = time.time()
    if cv2.waitKey(1) == 27:
        break

cam.release()
cv2.destroyAllWindows()

### Curl

In [7]:
# HARDCODE
current_workout = curl

# Initialize fps counter
fps_time = 0

# Load model
model = TfPoseEstimator(GRAPH_PATH, target_size=PROCESSING_DIMS)

# Initialize camera capture
image1 = cv2.imread(os.path.join(DATA_PATHS['sample'], 'curls', 'curl-up.png'))
image2 = cv2.imread(os.path.join(DATA_PATHS['sample'], 'curls', 'curl-down.png'))

# Predict poses
humans1 = model.inference(image1)
humans2 = model.inference(image2)

# Analyze workout
dev1 = analyze_workout(humans1[0].body_parts, 'L', current_workout)
dev2 = analyze_workout(humans2[0].body_parts, 'L', current_workout)

# Draw pose
draw1 = TfPoseEstimator.draw_humans(image1, humans1, imgcopy=False)
draw2 = TfPoseEstimator.draw_humans(image2, humans2, imgcopy=False)

# Draw angle
cv2.putText(draw1,
                "Deviation: {}".format(dev1),
                (10, 40),  cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0), 2)

cv2.putText(draw2,
                "Deviation: {}".format(dev2),
                (10, 40),  cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0, 255, 0), 2)

# Read loop
while True:
    
    # Show image
    cv2.imshow('tf-pose-estimation result1', draw1)
    cv2.imshow('tf-pose-estimation result2', draw2)

    # Restart FPS counter
    fps_time = time.time()
    if cv2.waitKey(1) == 27:
        break

cam.release()
cv2.destroyAllWindows()

D:\Softwares\Anaconda\envs\tensor_gpu\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars


## Live Test

### Helpers

In [8]:
def get_best_human(humans):
    # Get biggest human
    human = None
    largest_torso = 0
    for h in humans:
        try:
            # average shoulders
            shoulder = average_or_one(h.body_parts, 2, 5)
            # average hips
            hip = average_or_one(h.body_parts, 8, 11)
            
            if shoulder and hip:
                torso = (hip[0] - shoulder[0])**2 + (hip[1] - shoulder[1])**2
                torso = math.sqrt(torso)
                
                if torso > largest_torso:
                    largest_torso = torso
                    human = h
        except KeyError:
            pass
        
    return human

def draw_sizes(image, humans):
    imw, imh = image.shape[0], image.shape[1]
    human = None
    largest_torso = 0
    for h in humans:
        try:
            # average shoulders
            shoulder = average_or_one(h.body_parts, 2, 5)
            # average hips
            hip = average_or_one(h.body_parts, 8, 11)
            
            if shoulder and hip:
                torso = (hip[0] - shoulder[0])**2 + (hip[1] - shoulder[1])**2
                torso = math.sqrt(torso)
                
                if torso > largest_torso:
                    largest_torso = torso
                    human = h
                    cv2.putText(image,
                                "SIZE: {}".format(torso),
                                (int(shoulder[0]*imw-5), int(shoulder[1]*imh)),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (0, 0, 255), 2)
                else:
                    cv2.putText(image,
                                "SIZE: {}".format(torso),
                                (int(shoulder[0*imw]-5), int(shoulder[1]*imh)),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (0, 255, 0), 2)
        except KeyError:
            pass
        
    return image
        
    

In [9]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

### Plank

In [20]:
# HARDCODE
current_workout = plank

# Initialize fps counter
fps_time = 0

# Load model
model = TfPoseEstimator(GRAPH_PATH, target_size=PROCESSING_DIMS)

# Initialize camera capture
#cam = cv2.VideoCapture(0)
cam = cv2.VideoCapture('data/sample/plank.mp4')
#ret_val, image = cam.read()

#rows, cols = image.shape[0], image.shape[1]

# Read loop
while True:
    # Read from camera
        ret_val, image = cam.read()
        #image = imutils.rotate(image, 90)
        cv2.imshow('',image)
        while True:
            if cv2.waitKey(1) == 27:
                break

        # Predict poses
        humans = model.inference(image)

        subject = get_best_human(humans)

        try:
            # Analyze workout
            dev = analyze_workout(subject.body_parts,0, current_workout)
        except IndexError as e:
            image = imutils.rotate(image, -90)
            cv2.imshow('tf-pose-estimation result', image)
            fps_time = time.time()
            if cv2.waitKey(1) == 27:
                break
            continue


        # Draw pose
        draw = TfPoseEstimator.draw_humans(image, [subject], imgcopy=False)
        draw = imutils.rotate(draw, -90)

        # Draw angle
        cv2.putText(draw,
                        "Deviation: {}".format(dev),
                        (10, 40),  cv2.FONT_HERSHEY_SIMPLEX, 1,
                        (0, 255, 0), 2)

        # Draw fps
        fps = 1.0 / (time.time() - fps_time)
        cv2.putText(draw,
                    "FPS: {}".format(fps),
                    (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 0), 2)

        # Show image
        cv2.imshow('tf-pose-estimation result', draw)

        # Restart FPS counter
        fps_time = time.time()
        if cv2.waitKey(1) == 27:
            break

cam.release()
cv2.destroyAllWindows()

TypeError: 'bool' object is not subscriptable

### Curls

In [11]:
# HARDCODE
current_workout = curl

# Initialize fps counter
fps_time = 0

# Load model
model = TfPoseEstimator(GRAPH_PATH, target_size=PROCESSING_DIMS)

# Initialize camera capture
#cam = cv2.VideoCapture(CAMERA_NUMBER)
cam = cv2.VideoCapture('data/sample/curls.mp4')
# ret_val, image = cam.read()

# rows, cols = image.shape[0], image.shape[1]

# Read loop
while True:
    # Read from camera
    ret_val, image = cam.read()

    # Predict poses
    humans = model.inference(image)

    try:
        subject = get_best_human(humans)
        image = draw_sizes(image, humans)
        if subject == None:
            raise IndexError()
        # Analyze workout
        dev = analyze_workout(subject.body_parts, 'L', current_workout)
    except IndexError as e:
        cv2.imshow('tf-pose-estimation result', image)
        fps_time = time.time()
        if cv2.waitKey(1) == 27:
            break
        continue
        

    # Draw pose
    draw = TfPoseEstimator.draw_humans(image, [subject], imgcopy=False)

    # Draw angle
    cv2.putText(draw,
                    "Deviation: {}".format(dev),
                    (10, 40),  cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 255, 0), 2)

    # Draw fps
    fps = 1.0 / (time.time() - fps_time)
    cv2.putText(draw,
                "FPS: {}".format(fps),
                (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (0, 255, 0), 2)
    
    # Show image
    cv2.imshow('tf-pose-estimation result', draw)

    # Restart FPS counter
    fps_time = time.time()
#     time.sleep(1)
    if cv2.waitKey(1) == 27:
        break

cam.release()
cv2.destroyAllWindows()

D:\Softwares\Anaconda\envs\tensor_gpu\lib\site-packages\ipykernel_launcher.py:97: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\USER\Documents\phormatics\server\helpers\calculate_angle.py:13: RuntimeWarning: invalid value encountered in arccos
  return numpy.arccos(dot_product / norm_product)


### Pushup

In [15]:
# HARDCODE
current_workout = pushup

# Initialize fps counter
fps_time = 0

# Load model
model = TfPoseEstimator(GRAPH_PATH, target_size=PROCESSING_DIMS)

# Initialize camera capture
cam = cv2.VideoCapture(CAMERA_NUMBER)
ret_val, image = cam.read()

rows, cols = image.shape[0], image.shape[1]

# Read loop
while True:
    # Read from camera
    ret_val, image = cam.read()
    
    image = imutils.rotate(image, 90)

    # Predict poses
    humans = model.inference(image)

    try:
        subject = get_best_human(humans)
        image = draw_sizes(image, humans)
        if subject == None:
            raise IndexError()
        # Analyze workout
        dev = analyze_workout(subject.body_parts, None, current_workout)
    except IndexError as e:
        image = imutils.rotate(image, -90)
        cv2.imshow('tf-pose-estimation result', image)
        fps_time = time.time()
        if cv2.waitKey(1) == 27:
            break
        continue
        

    # Draw pose
    draw = TfPoseEstimator.draw_humans(image, [subject], imgcopy=False)
    
    image = imutils.rotate(draw, -90)

    # Draw angle
    cv2.putText(draw,
                    "Deviation: {}".format(dev),
                    (10, 40),  cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 255, 0), 2)

    # Draw fps
    fps = 1.0 / (time.time() - fps_time)
    cv2.putText(draw,
                "FPS: {}".format(fps),
                (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                (0, 255, 0), 2)
    
    # Show image
    cv2.imshow('tf-pose-estimation result', draw)

    # Restart FPS counter
    fps_time = time.time()
#     time.sleep(1)
    if cv2.waitKey(1) == 27:
        break

cam.release()
cv2.destroyAllWindows()

NotFoundError: /home/jasonchin/projects/phorm/phormatics/models/mobilenet/graph_opt.pb; No such file or directory

None
